# Time-dependent Schrödinger Equation

Consider the time-dependent one-dimensional quantum harmonic oscillator defined by the Hamiltonian:

$$
H = \frac{\hat{p}^2}{2m} + \frac{\omega^2}{2m} \left(\hat{q} - q_0(t)\right)^2
$$

where:
$$
q_0(t) = \frac{t}{T}, \quad t \in [0, T].
$$

The initial state is given by:
$$
|\Psi_0\rangle = |n = 0\rangle
$$
(the ground state of the harmonic oscillator).

Note: we will use the standard convention of $\hbar = m = 1$ .

## Objectives

1. Compute the time-evolved state $(|\Psi(t)\rangle)$ for different values of $(T)$.
2. Plot:
   - The square norm of $(|\Psi(t)\rangle)$ as a function of $(q)$ at different times.
   - The average position of the particle $(\langle q(t) \rangle)$ as a function of time $(t)$.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from numpy.polynomial.hermite import hermval
from numpy.linalg import norm
from scipy.special import factorial
from matplotlib import animation  # for creating gifs
from aux import *

## Solution ##

The analytical expression for the eigenfunctions for this Hamiltonian can be expressed through the Hermite polynomials as follows:

$$
Ψ_n(q; t) = \left(\frac{ω}{π}\right)^{1/4}\frac{1}{\sqrt{2^nn!}}
            \exp\left({\frac{-ω(q - q₀(t))^2}{2}}\right)
            H_n\left(\sqrt{ω} (q - q₀(t))\right)
$$

The necessary functions to implement it were imported before from aux.py .

# Objects inizialitation # 

Now, let's create classes for containing our operators and variables.

In [2]:
class Param:
    """
    Container for holding all simulation parameters

    Parameters
    ----------
    xmin, xmax : float
        The real space is between [xmin, xmax]
    Nx : int
        Number of intervals in [xmin, xmax]
    dt : float
        Time discretization
    Nt : int
        Number of timestep
    im_time : bool, optional
        If True, use imaginary time evolution.
        Default to False.
    """
    
    def __init__(self,
                 xmax: float,
                 xmin: float,
                 Nx: int,
                 tsim: float,
                 Nt: int,
                 omega: float,
                 im_time: bool = False) -> None:
    
        self.xmax = xmax
        self.xmin = xmin
        self.Nx = Nx
        self.tsim = tsim
        self.Nt = Nt
        self.omega = omega
        self.im_time = im_time
        
        # Derived parameters
        self.dx = (xmax - xmin) / Nx
        self.dt = tsim / Nt
        self.x = np.linspace(xmin + 0.5 * self.dx, xmax - 0.5 * self.dx, Nx)
        # Momentum, defined as 1/x and a prefactor from Fourier Transfrom
        self.dk = 2 * np.pi / (xmax - xmin)
        # Momentum grid
        self.k = np.fft.fftfreq(Nx, d=self.dx) * 2 * np.pi
        
class Operators:   
    """Container for holding operators and wavefunction coefficients."""
    
    def __init__(self, 
                 res: int,
                 par: Param,
                 voffset: float = 0.,
                 wfcoffset: float = 0.,
                 q0 = lambda q: 0,
                 n: int = 0) -> None:

        # Initialize the containers for the various operators
        self.V = np.empty(res, dtype=complex)
        self.R = np.empty(res, dtype=complex)
        self.K = np.empty(res, dtype=complex)
        self.wfc = np.empty(res, dtype=complex)
        
        
        # Time-dependent offset (default to zero)
        self.q0 = q0 or (lambda q: 0)
        
        # Initialize parameters and compute the energy
        if par is not None:
            self.init_operators(par, voffset, wfcoffset, n)
            self.compute_energy(par,)
            
    def init_operators(self, par: Param, voffset: float = 0., wfcoffset: float = 0., n: int = 0) -> None:
        
        # Initialize time-dependent offset
        q0 = self.q0(0)
        
        # Potential
        self.V = 0.5 * (par.omega ** 2) * (par.x - voffset - q0) **2
        self.wfc = decomposition(par.x - wfcoffset, n, par.omega).astype(complex)
        
        # Set the coefficient that regulates whether we're doing real or imaginary time evolution
        coeff = 1 if par.im_time else 1j    # for clarification, look at what done in the hands-on (ite_new.ipynb)
        
        ## Kinetic 
        ## We use Hausdorff approx (e^{A+B} \sim e^{A}*e^{B}+O(DeltaT^2))
        # In momentum space (kinetic part of the Hamiltonian)
        self.K = np.exp(-0.5 * (par.k **2) * par.dt * coeff)     # this is already the exponential term
        # In real space (potential part of the Hamiltonian)
        self.R = np.exp(-0.5 * self.V * par.dt * coeff)     # here U = e^{-V*t/2}e^{-K*t}e^{-v*t/2}+O{Deltat^3}   
        
    def compute_energy(self, par: Param) -> float:
    
        # Initialize the 3 wavefunctions: real space, momentum space, conjugate.
        wfc_r = self.wfc
        wfc_k = np.fft.fft(self.wfc)
        wfc_c = np.conj(self.wfc)
        
        # Compute both the real and momentum space energy terms
        E_r = wfc_c * self.R * wfc_r
        E_k = 0.5 * wfc_c * np.fft.ifft((par.k**2) * wfc_k) 
        
        # Sum over all space
        energy = (sum(E_r, E_k).real) * par.dx  # we take the real part since the energy is a real quantity by definition
        
        return energy
        

## Time evolution ##

Here we evolve our wavefunction in time, having the evolution operator (thanks Trotter-Suzuki approximation) as:
$$
\hat{U}(\Delta t) = e^{-i\hat{H}(t)\Delta t / \hbar} \\
                = e^{-i\hat{K}(t)\Delta t / \hbar} e^{-i\hat{V}(t)\Delta t / \hbar} + O(\Delta t^2) \\
                = e^{-i\hat{V}(t)\Delta t / 2\hbar} e^{-i\hat{K}(t)\Delta t / \hbar} e^{-i\hat{V}(t)\Delta t / 2\hbar} + O(\Delta t^3)
$$
The time evolution of the wavefunction for a time $t=N_t\Delta t$ is given by applying $N_t$ times the evolution operator, which equivaltes to:
$$
\Psi(x,t) = \hat{U}(\Delta t)^{N_t} \Psi(x,0)
$$
Then we move to the Fourier transform space and rewrite the evolution operator there.

In [3]:
def time_evolution(par: Param, opr: Operators, check_norm: bool=False) -> np.array: 
    """ Performs the time evolution of the wavefunction for a given number of timesteps."""
    
    # Store the density value of the wavefunction for each timestep (1st half for real, 2nd half for momentum)
    densities = np.zeros((par.Nt,2*par.Nx)) 
    potential = np.zeros((par.Nt,par.Nx))
    avg_pos = np.zeros((par.Nt,par.Nx))


    for t in range(par.Nt):
        # Apply the evolution operator Nt time = U^{Nt}
        
        # Check normalization of the wavefunction (only for the real time case)
        if not par.im_time and check_norm:
            check_normalization_wfc(opr.wfc, par.x)
        
        # Update the time-dependent potential
        q0 = opr.q0(t * par.dt)
        opr.V = 0.5 * (par.omega ** 2) * (par.x - q0) **2
        
        # Update the real operator
        coeff = 1 if par.im_time else 1j
        opr.R = np.exp(-0.5 * opr.V * par.dt * coeff)
        
        # Apply the 'first half' of the potential part (real space)
        opr.wfc *= opr.R
        
        # Check normalization of the wavefunction (only for the real time case)
        if not par.im_time and check_norm:
            check_normalization_wfc(opr.wfc, par.x)
        
        # Move to momentum space
        opr.wfc = np.fft.fft(opr.wfc)

        # Kinetic part
        opr.wfc *= opr.K

        # Back to real space
        opr.wfc = np.fft.ifft(opr.wfc)
        
        # Check normalization of the wavefunction (only for the real time case)
        if not par.im_time and check_norm:
            check_normalization_wfc(opr.wfc, par.x)
        
        # 'Second half' of the potential part
        opr.wfc *= opr.R
        
        # Check normalization of the wavefunction (only for the real time case)
        if not par.im_time and check_norm:
            check_normalization_wfc(opr.wfc, par.x)
        
        # Density |Psi|^2
        density =  np.abs(opr.wfc)**2
        
        # Renormalization for imaginary time (here the evolution operator is not unitary anymore)
        if par.im_time:
            norm = np.sum(density * par.dx)
            opr.wfc /= np.sqrt(norm)
            density = np.abs(opr.wfc)**2
                
        densities[t, 0:par.Nx] = np.real(density)   # real space
        densities[t, par.Nx:2 * par.Nx] = np.abs(np.fft.fft(opr.wfc)) ** 2   #momentum space
        
        # Save the potential
        potential[t, :] = opr.V
        
        # Save the average position
        avg_pos[t] = np.sum(par.x * density) * par.dx
        
    return densities, potential, avg_pos
        

## Testing ##

In [ ]:
# SIMULATION PARAMETERS

xmax = 5.0
xmin = -5.0
Nx = 4000
tsim = 1000
desired_dt = 0.01
Nt = int(tsim / desired_dt)
im_time = False
omega = 1.0


# Initialize the Param object
par = Param(xmax, xmin, Nx, tsim, Nt, omega, im_time)

# ===========================================================================================================

# INITIAL CONDITIONS

res = Nx
voffset = 0.0
wfcoffset = 0.0
n = 0
q0_func = lambda t: t / tsim 

# Initialize the Operators object
opr = Operators(res, par, voffset, wfcoffset, q0_func, n)

print(f"The final energy of the systems is: {opr.compute_energy(par)[-1]}")

# Run the simulation and get the results
space_str = 'Imaginary' if par.im_time else 'Real'
print('Space:', space_str)
density, potential, avg_position = time_evolution(par, opr, check_norm=True)

The final energy of the systems is: 0.999997656238062
Space: Real


In [ ]:
# plot average position
plt.figure(figsize=(10, 5))
plt.plot(avg_position)
plt.title(f'Average position evolution in {space_str} time(ω={par.omega},T={par.tsim})')
plt.xlabel('t')
plt.ylabel('Position(x)')
plt.grid()
plt.savefig(f'avg_pos_{space_str}_timeT{par.tsim}.png')
plt.show()

# Plot potential as a heatmap
plt.figure(figsize=(10, 5))
plt.imshow(potential.T, extent=[0, tsim, xmin, xmax], aspect='auto', origin='lower', cmap='viridis')
plt.colorbar(label='Potential')
plt.title(f'Potential evolution in {space_str} time (ω={par.omega}, T={par.tsim})')
plt.xlabel('t')
plt.ylabel('Position (x)')
plt.grid()
plt.savefig(f'potential_heatmap_{space_str}_timeT{par.tsim}.png')
plt.show()


In [ ]:
print('Space:', space_str)

# Generate the data
density, potential, avg_position = time_evolution(par, opr)

# Set up the figure and axis for the animation
step_interval = max(par.Nt // 100, 1)  # Ensure at least 200 frames, adjust as needed
selected_frames = range(5, par.Nt, step_interval)

fig, ax = plt.subplots()
ax.set_xlim(-par.xmax, par.xmax)
ax.set_ylim(0, 1)
ax.set_xlabel("Position (x)")
ax.set_ylabel("Probability Density |ψ(x)|^2")
ax.set_title(f'{space_str} Space evolution (ω={par.omega}, T={par.tsim})')

# Plot the potential and prepare the line for the wave function
line_wfc, = ax.plot([], [], lw=2, label="Wave Function |ψ(x)|^2", color='blue')
line_pot, = ax.plot([], [], label="Potential V(x)", color='gray', linestyle='--')
line_avg, = ax.plot([], [], label="Average position", color='red', linestyle='--')
ax.legend()
ax.grid()

# Initialization function for the animation
def init_line():
    line_wfc.set_data([], [])
    line_pot.set_data([], [])
    line_avg.set_data([], [])
    
    return line_wfc, line_pot, line_avg

# Animation function that updates the plot at each frame
def animate(i):
    x = par.x
    y_wfc = density[i, :par.Nx]
    y_pot = potential[i, :]
    x_avg = avg_position[i,0]
    
    line_wfc.set_data(x, y_wfc)
    line_pot.set_data(x, y_pot)
    line_avg.set_data([x_avg,x_avg], [0, 1.5])
    
    return line_wfc, line_pot, line_avg

# Create the animation
anim = animation.FuncAnimation(
    fig, animate, init_func=init_line, frames=selected_frames, interval=10, blit=True
)

# Save the animation as a GIF
writer = animation.PillowWriter(fps=10, metadata=dict(artist='Me'), bitrate=1800)
anim.save(f'{space_str}_spaceT{par.tsim}.gif', writer=writer)

plt.show()